In [2]:
with open('input.txt' , 'r', encoding = 'utf-8') as f:
        text = f.read()

In [3]:
len(text)

1115394

In [4]:
text[:10]

'First Citi'

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


In [7]:
vocab_size

65

In [8]:
chars

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [22]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [23]:
a = encode("hi")
a

[46, 47]

In [25]:
print(decode(a))

hi


In [26]:
import torch
data = torch.tensor(encode(text),dtype = torch.long)

In [27]:
print(data.shape)

torch.Size([1115394])


In [29]:
data.dtype

torch.int64

In [30]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [32]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data [n:]

In [33]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [35]:
torch.manual_seed(1337)
batch_size = 4
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size] for i in ix])
    return x,y

In [36]:
 xb,yb = get_batch('train')

In [ ]:
xb,yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43],
         [53, 56,  1, 58, 46, 39, 58],
         [58,  1, 58, 46, 39, 58,  1],
         [17, 27, 10,  0, 21,  1, 54]]))

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,input_index,targets):
        logits = self.token_embedding_table(input_index)
        return logits

In [44]:
m = BigramModel(vocab_size=vocab_size)
out = m(xb,yb)
print(out.shape)

torch.Size([4, 8, 65])


tensor([[-0.2103,  0.4481,  1.2381,  0.6091,  0.4209, -1.3998, -0.4007, -0.1462,
         -1.1387, -0.0134, -1.9390,  0.6582,  0.6734,  0.7523, -0.7533, -0.1611,
          0.8354,  1.7862, -0.7115,  0.2381, -1.2085,  0.0717,  0.1532, -2.2039,
          0.6705,  0.2791, -0.2735,  0.3476, -0.2101,  2.2659,  1.0809, -0.4287,
         -0.2424, -0.2258, -0.3756,  0.3991,  0.7816,  1.2656,  1.3015,  0.1370,
          0.9264, -1.1001, -1.0641,  0.6243, -0.2436,  0.1079,  0.4256,  0.1838,
         -0.1147,  1.7112,  1.2666,  0.7957,  0.9475,  1.3239,  2.0084,  1.4341,
         -0.4606, -0.0487, -1.1202,  1.3071,  0.0508,  0.2770,  1.3597, -0.0821,
          0.3909],
        [ 0.2475, -0.6349, -1.2909,  1.1822,  0.1479, -0.4333, -0.8269,  0.0728,
         -1.2982,  0.3960, -1.2460,  0.1458, -0.5699, -1.3561, -0.3812, -0.8515,
          1.1918, -0.8108, -0.1733, -0.4703, -0.6000, -1.3636, -1.0889,  1.0108,
          0.8543, -0.0441,  1.8017,  0.6014, -2.5448, -0.4865,  2.6412,  1.6053,
         